In [14]:
import scipy.io as sio
import tensorflow as tf
import larq as lq
import numpy as np
from scipy.special import logsumexp

In [15]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(np.shape(train_images))

(60000, 28, 28)


In [16]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 255, test_images / 255

In [4]:
sio.savemat('data.mat', {'train_images':train_images, 'train_labels':train_labels,
                              'test_images':test_images, 'test_labels':test_labels})

In [41]:
thresholdVal = 0.05
layerSize = 100
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer=lq.quantizers.SteTern(threshold_value=thresholdVal,
                                ternary_weight_networks=False,
                                clip_value=1.0),
              kernel_constraint="weight_clip")
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    lq.layers.QuantDense(layerSize, kernel_quantizer=lq.quantizers.SteTern(threshold_value=0.05,
                                                    ternary_weight_networks=False,
                                                    clip_value=1.0),
                                kernel_constraint="weight_clip",
                                use_bias=False),
    tf.keras.layers.BatchNormalization(scale=False, center=False),
    lq.layers.QuantDense(layerSize, use_bias=False, **kwargs),
    tf.keras.layers.BatchNormalization(scale=False, center=False),
    lq.layers.QuantDense(10, use_bias=False, **kwargs),
    #tf.keras.layers.BatchNormalization(scale=False, center=False),
    tf.keras.layers.Activation("softmax")])

In [42]:
lq.models.summary(model3)

+sequential_4 stats---------------------------------------------------------------------------------+
| Layer                  Input prec.    Outputs  # 2-bit  # 32-bit  Memory  2-bit MACs  32-bit MACs |
|                              (bit)                 x 1       x 1    (kB)                          |
+---------------------------------------------------------------------------------------------------+
| flatten_4                        -  (-1, 784)        0         0       0           0            0 |
| quant_dense_12                   -  (-1, 100)    78400         0   19.14           0        78400 |
| batch_normalization_8            -  (-1, 100)        0       200    0.78           0            0 |
| quant_dense_13                   1  (-1, 100)    10000         0    2.44       10000            0 |
| batch_normalization_9            -  (-1, 100)        0       200    0.78           0            0 |
| quant_dense_14                   1   (-1, 10)     1000         0    0.24        

In [44]:
# tf.random.set_seed(2020)
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(train_images, train_labels, batch_size=64, epochs=50)
test_loss, test_acc = model3.evaluate(test_images, test_labels, verbose=2)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 7s 114us/sample - loss: 0.3183 - accuracy: 0.9496
Epoch 2/50
60000/60000 [==============================] - 8s 126us/sample - loss: 0.3035 - accuracy: 0.9511
Epoch 3/50
60000/60000 [==============================] - 6s 106us/sample - loss: 0.3076 - accuracy: 0.9497
Epoch 4/50
60000/60000 [==============================] - 5s 91us/sample - loss: 0.3047 - accuracy: 0.9506
Epoch 5/50
60000/60000 [==============================] - 7s 120us/sample - loss: 0.3017 - accuracy: 0.9507
Epoch 6/50
60000/60000 [==============================] - 6s 100us/sample - loss: 0.2990 - accuracy: 0.9513
Epoch 7/50
60000/60000 [==============================] - 8s 141us/sample - loss: 0.2870 - accuracy: 0.9527
Epoch 8/50
60000/60000 [==============================] - 19s 310us/sample - loss: 0.2978 - accuracy: 0.9520
Epoch 9/50
60000/60000 [==============================] - 9s 150us/sample - loss: 0.2962 - accuracy: 0.9509
Epoch

In [45]:
def SteTernWeight(weight, threshold=0.05):
    (m, n) = np.shape(weight)
    for i in range(m):
        for j in range(n):
            if (weight[i, j] >= 0.05):
                weight[i,j] = 1
            elif (weight[i, j] >= -0.05):
                weight[i, j] = 0
            else:
                weight[i, j] = -1
    return weight
                

def SignVec(vec):
    vec1 = (vec >= 0)
    vec1 = 2 * vec1 - 1
    return vec1

In [46]:
testData = test_images[0, :, :, :].reshape(28*28,1)
weight1 = model3.layers[1].get_weights()[0]
weight2 = model3.layers[3].get_weights()[0]
weight3 = model3.layers[5].get_weights()[0]

mean1 = model3.layers[2].get_weights()[0].reshape(layerSize, 1)
# offset1 = model3.layers[2].get_weights()[1].reshape(layerSize, 1)
variance1 = model3.layers[2].get_weights()[1].reshape(layerSize, 1)

mean2 = model3.layers[4].get_weights()[0].reshape(layerSize, 1)
# offset2 = model3.layers[4].get_weights()[1].reshape(layerSize, 1)
variance2 = model3.layers[4].get_weights()[1].reshape(layerSize, 1)

weight1 = SteTernWeight(weight1, threshold=thresholdVal)
weight2 = SteTernWeight(weight2, threshold=thresholdVal)
weight3 = SteTernWeight(weight3, threshold=thresholdVal)

l1 = np.matrix(weight1).T * testData
l1 = (np.array(l1) - mean1) #/ np.sqrt(variance1)
#l1 = np.sqrt(variance1) * np.array(l1) + mean1
l1 = SignVec(l1)

l2 = np.matrix(weight2).T * l1
l2 = (np.array(l2) - mean2) #/ np.sqrt(variance2)
#l2 = np.sqrt(variance2) * np.array(l2) + mean2
l2 = SignVec(l2)

l3 = np.matrix(weight3).T * l2

print(l3)

print(np.exp(l3-logsumexp(l3)))

[[ -1.]
 [  5.]
 [  3.]
 [  6.]
 [  0.]
 [ -5.]
 [ -8.]
 [ 33.]
 [-20.]
 [  0.]]
[[1.71390843e-15]
 [6.91440011e-13]
 [9.35762297e-14]
 [1.87952882e-12]
 [4.65888615e-15]
 [3.13913279e-17]
 [1.56288219e-18]
 [1.00000000e+00]
 [9.60268005e-24]
 [4.65888615e-15]]


In [48]:
testData = test_images[0, :, :, :].reshape(28*28, order="A")
print(testData[200:240])

[0.         0.         0.32941176 0.7254902  0.62352941 0.59215686
 0.23529412 0.14117647 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.87058824 0.99607843 0.99607843 0.99607843 0.99607843 0.94509804
 0.77647059 0.77647059 0.77647059 0.77647059]


In [49]:
np.sum(weight1 == 0)/ np.size(weight1)

0.1860969387755102

In [50]:
np.sum(weight2 == 0) / np.size(weight2)

0.1991

In [51]:
np.sum(weight3 == 0) / np.size(weight3)

0.471

In [52]:
model3.predict(test_images[0:1, :, :, :])

array([[1.7139086e-15, 6.9144002e-13, 9.3576222e-14, 1.8795289e-12,
        4.6588866e-15, 3.1391329e-17, 1.5628822e-18, 1.0000000e+00,
        9.6026815e-24, 4.6588866e-15]], dtype=float32)

In [53]:
mean3 = np.zeros(np.shape(weight3)[1])
upper = np.ones(np.shape(weight1)[0])
lower = np.zeros(np.shape(weight1)[0])
modelToSave = [{"name":"Flatten1", "type":"flatten", "inputSize": (28,28,1)},
               {"name":"QuantDense1", "type":"dense", "weights": weight1.T, "bias": -mean1,
                "activation":"Sign","upper":upper, "lower":lower}, 
               {"name":"QuantDense2", "type":"denseBin", "weights": weight2.T, "bias": -mean2, "activation":"Sign"},
               {"name":"Dense", "type":"denseBin", "weights": weight3.T, "bias": mean3}]
sio.savemat('nn2x100.mat', {'nn':modelToSave})
#print(modelToSave[1])

In [37]:
mean3

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])